## Rekognisi Karakter OCR pada PDF
Tujuan dari project ini adalah untuk mengubah konten dari pdf hasil scan hardcopy menjadi file text. Jumlah file yang harus dikonversi adalah 79 ribu file. Opsi pengerjaan dilakukan dengan menggunakan VPS dengan spesifikasi sebagai berikut:
- RAM 12 GB
- Processor Intel Xeon
- SSD 60 GB
- OS Ubuntu 22.04 LTS

In [3]:
# Kumpulkan nama file dalam folder tugas dalam satu list
import os

## Setting up file path
path = os.path.expanduser('~/local-drive/folder-target-pekerjaan')

file_list = os.listdir(path)
file_list = file_list[1:]
print(len(file_list))
print(file_list[:20])

72305
['000918.16.2023w.pdf', '010593.16.2020w.pdf', '014528.27.2022w.pdf', '001824.47.2024w.pdf', '008588.16.2023w.pdf', '000914.16.2023w.pdf', '000923.16.2023w.pdf', '001237.16.2023w.pdf', '002103.16.2023w.pdf', '000915.16.2023w.pdf', '008581.16.2023w.pdf', 'PUTP1 005015.16.2022w.pdf', '000920.16.2023w.pdf', '008580.16.2023w.pdf', '001955.12.2023w.pdf', '000994.15.2020w.pdf', '000707.45.2024w.pdf', 'PUTP1 005638.15.2022w.pdf', '010386.25.2023w.pdf', '001951.25.2023w.pdf']


In [5]:
## Import libraries untuk OCR
from pdf2image import convert_from_path
import numpy as np
import cv2
import easyocr

## Setting output path
output = os.path.expanduser('~/local-drive/hasil-pekerjaan')

In [6]:
# Preparing the grayscale function
def preprocessing(img):
  gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
  gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
  gray = cv2.medianBlur(gray, 3)
  gray = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)
  return gray

In [7]:
# Setup reader
reader = easyocr.Reader(['id', 'en'], gpu = False)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
# Definisikan batch_size
batch_size = 500

# loop per batch
for batch_start in range(0, len(file_list), batch_size):
    batch_end = batch_start + batch_size
    batch_files = file_list[batch_start:batch_end]

    print(f"🔄 Memproses batch {batch_start} - {batch_end-1} "
          f"(total {len(batch_files)} file)")
    
    for f in batch_files:
        file_name = os.path.join(path, f)
        
        pages = convert_from_path(file_name, dpi=300)
        all_text = []
        
        # OCR for each Pages
        for page in pages:
            img = preprocessing(img=page)
            result = reader.readtext(img, detail = 0)
            
            # gabungkan kata menjadi satu paragraf
            text = " ".join(result)
            
            # bersihkan spasi berlebih
            text = " ".join(text.split())
            all_text.append(text)
        output_file = os.path.join(output, f'{f[:-4]}.txt')
        
        with open(output_file, 'w', encoding='utf-8') as o:
            o.write("\n\n".join(all_text))

🔄 Memproses batch 0 - 499 (total 500 file)


/home/alchemista/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
